In [112]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [113]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [114]:
demo = pd.read_csv("PTDEMOG.csv").rename(columns={"PHASE":"Phase"})

In [115]:
neuro = pd.read_csv("NEUROEXM.csv").rename(columns={"PHASE":"Phase"})

In [116]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [117]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [118]:
clinical.head()

,Phase,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXODES,DXCONFID,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,2,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00.0
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,-4.0,3.0,4,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00.0
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,6,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00.0
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,3.0,8,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00.0
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,-4.0,4.0,10,10,2005-10-06,NaN,NaN,NaN,NaN,2005-10-06 00:00:00.0


In [132]:
# diag["Subject"].value_counts()

In [120]:
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv").rename(columns={"PHASE":"Phase"})[["RID", "PTID" , "Phase"]]

In [121]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"])

In [122]:
m = m.rename(columns={"PTID_x":"PTID"})
m = m.drop(["PTID_y"], axis=1)


In [123]:
m = m.merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [124]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [125]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [126]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [127]:
m["PTID"].value_counts()

PTID
067_S_4767     42
067_S_4072     32
141_S_0697     30
131_S_0384     29
116_S_0752     28
               ..
094_S_6122      1
011_S_6130      1
051_S_1338      1
020_S_6140      1
033_S_10107     1
Name: count, Length: 3022, dtype: int64

In [133]:
t = m
t.head()

,RID,PTID,Phase,VISCODE,VISCODE2,VISDATE,PTSOURCE,PTGENDER,PTDOB,PTDOBYY,...,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXODES,DXCONFID,GroupN,DIAGNOSIS_y
0,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0,1.0,1.0
1,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0,1.0,1.0
2,2,011_S_0002,ADNI1,sc,sc,2005-08-17,1.0,1.0,04/1931,1931.0,...,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0,1.0,1.0
3,3,011_S_0003,ADNI1,sc,sc,2005-08-18,1.0,1.0,05/1924,1924.0,...,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0,2.0,3.0
4,3,011_S_0003,ADNI1,sc,sc,2005-08-18,1.0,1.0,05/1924,1924.0,...,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0,2.0,3.0


In [129]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOBMM","DX"], axis=1) 

KeyError: "['PTDOBMM', 'DX'] not found in axis"

In [ ]:
t.columns

In [ ]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [ ]:
len(t.columns)

In [ ]:
t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
                                                                                                                   "bookkeeper").str.replace("cpa", "accounting")

In [ ]:
t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [ ]:
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

AttributeError: Can only use .str accessor with string values!

In [ ]:
cond = t['PTWORK'].value_counts()
threshold = 10
t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [ ]:
categorical = ['PTGENDER', 'PTWORK',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [ ]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [ ]:
text = ["PTWORK", "CMMED"]

In [ ]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

In [ ]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

In [ ]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]
t = t.drop(to_del, axis=1)

In [ ]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

In [ ]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [ ]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [ ]:
t[l]

In [ ]:
dfs = []

In [ ]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [ ]:
cat = pd.concat(dfs, axis=1)

In [ ]:
t[quant]

In [ ]:
cat

In [ ]:
t[["PTID","RID", "Phase", "Group"]]

In [ ]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [ ]:
c

In [ ]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [ ]:
c.to_csv("clinical.csv")

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.to_pickle("X_train_c.pkl")
y_train.to_pickle("y_train_c.pkl")

X_test.to_pickle("X_test_c.pkl")
y_test.to_pickle("y_test_c.pkl")